# Introduction to logistic regression
Logistic regression is one of the most basic and popular machine learning algorithms used to predict the probability of a class and classify given the values of different independent predictor variables.
The dependent variable (Y) is binary ,that is,it can take only two possible values 0 or 1. Example:If the objective is to determine  

In [1]:
using Pkg
Pkg.add("StatsBase")
Pkg.add("ClassImbalance")
Pkg.add("ROCAnalysis")
using Pkg
using DataFrames
using CSV
using Plots
using GLM
using StatsBase
using Lathe
using MLBase
using ClassImbalance
using ROCAnalysis
#enable printing of 1000 columns

ENV["COLUMNS"] = 1000 


   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
Updating `~/.julia/environments/v1.5/Project.toml`
  [2913bbd2] + StatsBase v0.33.5
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
  Installed Lathe ────────────────── v0.0.9
  Installed KernelDensity ────────── v0.5.1
  Installed PositiveFactorizations ─ v0.2.4
  Installed FillArrays ───────────── v0.8.14
  Installed ClassImbalance ───────── v0.8.7
  Installed StatsBase ────────────── v0.32.2
  Installed Optim ────────────────── v0.22.0
  Installed DataFrames ───────────── v0.20.2
  Installed PDMats ───────────────── v0.9.12
  Installed NLSolversBase ────────── v7.5.0
  Installed Tables ───────────────── v1.4.2
  Installed Parameters ───────────── v0.12.2
  Installed ArrayInterface ───────── v2.14.17
  Installed DiffResults ──────────── v1.0.3
  Installed UnPack ─────────────────

1000

In [2]:
#Read the file using CSV.File and convert it to dataframe
df = DataFrame(CSV.File("Churn_Modelling.csv"))
first(df,5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
,Int64,Int64,String,Int64,String,String,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64
1,1,15634602,Hargrave,619,France,Female,42,2,0.0,1,1,1,1.01349e5,1
2,2,15647311,Hill,608,Spain,Female,41,1,83807.9,1,0,1,1.12543e5,0
3,3,15619304,Onio,502,France,Female,42,8,1.59661e5,3,1,0,1.13932e5,1
4,4,15701354,Boni,699,France,Female,39,1,0.0,2,0,0,93826.6,0
5,5,15737888,Mitchell,850,Spain,Female,43,2,1.25511e5,1,1,1,79084.1,0


In [5]:
println(size(df))
describe(df)

(10000, 14)


,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Nothing,DataType
1,RowNumber,5000.5,1,5000.5,10000,,,Int64
2,CustomerId,1.56909e7,15565701,1.56907e7,15815690,,,Int64
3,Surname,,Abazu,,Zuyeva,2932,,String
4,CreditScore,650.529,350,652.0,850,,,Int64
5,Geography,,France,,Spain,3,,String
6,Gender,,Female,,Male,2,,String
7,Age,38.9218,18,37.0,92,,,Int64
8,Tenure,5.0128,0,5.0,10,,,Int64
9,Balance,76485.9,0.0,97198.5,2.50898e5,,,Float64


In [3]:
names(df)

14-element Array{Symbol,1}:
 :RowNumber
 :CustomerId
 :Surname
 :CreditScore
 :Geography
 :Gender
 :Age
 :Tenure
 :Balance
 :NumOfProducts
 :HasCrCard
 :IsActiveMember
 :EstimatedSalary
 :Exited

In [4]:
#Count the classes
countmap(df.Exited)
#the countmap function counts the number of unique entries in the dataframes column

Dict{Int64,Int64} with 2 entries:
  0 => 7963
  1 => 2037

# One Hot Encoding
One hot encoding is a process of converting categorical variables into a form (numerical columns) that could be fed into ML algorithms to do a better job in prediction 
3 categorical columns are present in the dataset. these columns 

In [6]:
Lathe.preprocess.OneHotEncode(df,:Geography)
Lathe.preprocess.OneHotEncode(df,:Gender)
select!(df, Not([:RowNumber, :CustomerId, :Surname, :Gender, :Male]))
first(df,5)

,CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,France,Spain,Germany,Female
,Int64,String,Int64,Int64,Float64,Int64,Int64,Int64,Float64,Int64,Bool,Bool,Bool,Bool
1,619,France,42,2,0.0,1,1,1,1.01349e5,1,1,0,0,1
2,608,Spain,41,1,83807.9,1,0,1,1.12543e5,0,0,1,0,1
3,502,France,42,8,1.59661e5,3,1,0,1.13932e5,1,1,0,0,1
4,699,France,39,1,0.0,2,0,0,93826.6,0,1,0,0,1
5,850,Spain,43,2,1.25511e5,1,1,1,79084.1,0,0,1,0,1


In [8]:
using Lathe.preprocess: TrainTestSplit
train, test = TrainTestSplit(df,.75);

In [10]:
fm = @formula(Exited ~ CreditScore + Age + Tenure + Balance + NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France + Spain)
logit = glm(fm, train, Binomial(), ProbitLink())

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Array{Float64,1},Binomial{Float64},ProbitLink},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

Exited ~ 1 + CreditScore + Age + Tenure + Balance + NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France + Spain

Coefficients:
─────────────────────────────────────────────────────────────────────────────────────
                       Coef.  Std. Error       z  Pr(>|z|)     Lower 95%    Upper 95%
─────────────────────────────────────────────────────────────────────────────────────
(Intercept)      -1.86359     0.168688    -11.05    <1e-27  -2.19422      -1.53297
CreditScore      -0.00041384  0.00018556   -2.23    0.0257  -0.000777531  -5.01501e-5
Age               0.0416653   0.00169614   24.56    <1e-99   0.0383409     0.0449897
Tenure           -0.00466681  0.00621166   -0.75    0.4525  -0.0168414     0.00750783
Balance           1.67258e-6  3.37154

In [11]:
prediction = predict(logit,test)

2553-element Array{Union{Missing, Float64},1}:
 0.16166540226873832
 0.3300210856851292
 0.2489955388111412
 0.051992624922222924
 0.16303633858773486
 0.1432134280608055
 0.08431562589429441
 0.33715902192170877
 0.3567413000629249
 0.42694002407534465
 0.05921422894114073
 0.04061726019236213
 0.5192273239291169
 ⋮
 0.018837917778999323
 0.04864939869721193
 0.18456629368330796
 0.20502948058211215
 0.15507332725345807
 0.043811398406759745
 0.1493081725538491
 0.04330450109157335
 0.503374957806429
 0.12235443103220423
 0.07256228547532302
 0.1473354850780677